## As the main goal of this challenge, you are expected to create a hash function based on quantum simulation. Your hash function's performance evaluation will be based on the following criteria:

1. Output determinism
2. Preservation of entropy
3. Computational difficulty
4. Preimage resistance
5. Collision resistance
6. Computational feasibility
7. Computation time
8. Purely quantum hashing

Their meaning will be demonstrated on a simple (and very bad) hash function.

The following hash function uses one qubit per one byte of input and applies the RX gates with the angles proportional to the bytes' values:

In [ ]:
pip install qiskit qiskit_ibm_runtime numpy

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Pauli, Statevector
import numpy as np

def simple_quantum_hash(input_bytes: bytes):
    num_qubits = len(input_bytes)
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        angle = (input_bytes[i] / 255) * np.pi  # scale to [0, π]
        qc.rx(angle, i)

    sv = Statevector.from_instruction(qc)
    exp_vals = [sv.expectation_value(Pauli("Z"), [i]).real for i in range(num_qubits)]

    # Map each expectation value from [-1, 1] to an 8-bit integer in [0, 255].
    output_bytes = bytearray([min(int(((val + 1) / 2) * 256), 255) for val in exp_vals])

    return output_bytes

At the very least, this function meets 2 of our most straightforward requirements. Firstly, it consistently produces the same output for a given input, satisfying the 'Output determinism' constraint, and, secondly, it does not use any classical hashing algorithms as required by the 'Purely classical hashing' point.

Let's now see what output our hash function produces for some simple inputs:

In [ ]:
print(list(simple_quantum_hash(bytes(range(0, 260, 20)))))

[255, 252, 240, 222, 198, 170, 139, 108, 78, 50, 28, 11, 2]


As you might've already deduced from the function definition, it basically calculates shifted and quantized cosine value for each of the input bytes. Unfortunately this fails to pass the 'Computational difficulty' requirement, as it is trivial to implement the same function without using any quantum simulators and such function would run in linear time with respect to the input length. This also makes finding the preimage of a given hash output an easy task, making it a cryptographically-poor hash function that is non-compliant with the 'Preimage resistance' criteria.

Since the cosine itself is a bijection from the domain of [0, π] to the codomain of [-1, 1] the hash collisions can only be possible due to the quantization. The easiest way to find such collisions is to look at the values closest to the minimum and maximum of a single byte range, where the derivative of the cosine is small:

In [ ]:
print(list(simple_quantum_hash(bytes(range(0, 20)))))
print(list(simple_quantum_hash(bytes(range(236, 256)))))

[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 254, 254, 254, 254, 253, 253, 253, 252, 252]
[3, 3, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


As you can see, there is plenty of byte collisions to be found in those regions. As a result, using this hash algorithm would result in a higher collision rate when compared to its classical counterparts, making it unsuitable for production applications and failing the 'Collision resistance' constraint.

The last non-trivial requirement is the 'Preservation of entropy', which our function, yet again, does not pass. The reason for that is simple - the cosine is not a linear function. That means that feeding a large set of randomly-generated inputs to our function would result in its output bytes being more highly concentrated towards their maximum and minimum values (as was previously demonstrated) and failing to behave like a sample from a random distribution over the function's codomain. Furthermore, some of the byte values are not possible to obtain at all due to the quantization:

In [ ]:
print(list(simple_quantum_hash(bytes(range(120, 135)))))

[139, 138, 136, 135, 133, 131, 130, 128, 127, 125, 124, 122, 120, 119, 117]


Lastly, the provided hash function does not pass the 'Computational feasibility' requirement, since it would require to simulate a 32-qubit circuit to calculate a 256-bit hash, which is not feasible to do on general-purpose hardware. This also makes the 'Computation time' criteria irrelevant, even thogh the function only employs one gate per qubit which would lead to good performance if the number of qubits was decreased.

In [ ]:
from qiskit.visualization import plot_state_city
from qiskit.quantum_info import Operator
from qiskit.circuit.library import UnitaryGate
from qiskit.quantum_info import random_unitary

#The increment part of the walker(if the coin is 1)
#The control qubit in the original circuit should be qubit n, while the walker register is everything before n
def inc(n:int):
  qc = QuantumCircuit(n) #N is the number of position qubits in the hashing function
  i = 0
  while(i<n-1):
    controls:list = list(range(i+1,n))
    qc.mcx(controls,i)
    i+=1
  qc.x(i)

  unitary = Operator(qc).data
  return UnitaryGate(unitary, 'Inc').control(num_ctrl_qubits=1)#Add with QuantumCircuit.apppend([]), control qubit is on the first in the list

#The decrement part of the walker(if the coin is 0)
def dec(n:int):
  qc = QuantumCircuit(n)
  i = 0
  while(i<n-1):
    controls:list = list(range(i+1,n))
    qc.mcx(controls,i,ctrl_state=0)
    i+=1
  qc.x(i)

  unitary = Operator(qc).data
  return UnitaryGate(Operator(qc).data, label="Shift")

#Must by laid out exactly like in the paper, where position comes first, then coin
def walk(n: int):
    from qiskit.quantum_info import Operator
    qc = QuantumCircuit(n + 1)  # pos[0]...pos[n-1], coin
    qc.append(inc(n), [n] + list(range(n)))  # INC if coin = 1
    qc.append(dec(n), [n] + list(range(n)))  # DEC if coin = 0
    unitary = Operator(qc).data
    return UnitaryGate(unitary, label="Shift")  # ❌ do not .control(n)


def quantum_walk_example():
  qc = QuantumCircuit(4)
  qc.h(3)
  #qc.cx([3,2,1],0)
  #qc.cx([3,2], 1)
  #qc.cx([3],2)
  #qc.cx([3,2,1],0, ctrl_state=0)
  #qc.cx([3,2], 1, ctrl_state=0)
  #qc.cx([3],2, ctrl_state=0)
  qc.append(inc(3),[3,0,1,2])
  qc.append(dec(3),[3,0,1,2])
  qc.h(3)

  sv = Statevector.from_instruction(qc)
  sc = plot_state_city(sv)
  #print(sv)
  sc.show()
  pass
quantum_walk_example()

ModuleNotFoundError: No module named 'qiskit'

In [ ]:
def create_UR_circuit(pos_qubits, seed=None):
    #qc = QuantumCircuit(pos_qubits, coin_qubit)
    n = len(pos_qubits)
    ur = random_unitary(2**n, seed=seed).to_gate(label="U_R")
    ur_c = ur.control(n)
    return ur_c

In [ ]:
import numpy as np
def U_mi(theta_map, m_i, i, pos_qubits, coin_qubit, seed=None):
    #m_i = msg_bits[2 * i:2 * i + 2]
    theta = theta_map[m_i]
    #unitary = coin_operator(theta, 'U_m'+str(i))
    #msg_val = int(m_i, 2)
    gate = coin_operator(theta, label="U_m"+str(i))
    controlled_gate = gate.control(2)
    #For coin qubit, define a 2x2 operation with random angle for [cos(theta), sin(theta), sin(theta), -cos(theta)] to the coin qubit
    #gate = UnitaryGate(coin_operator(theta, 'C'), label = "U_mi").control(2)
    return controlled_gate

In [ ]:
def coin_operator(theta, label='C'):
    C = np.array([
        [np.cos(theta), np.sin(theta)],
        [np.sin(theta), -np.cos(theta)]
    ])
    return UnitaryGate(C, label=label)

In [ ]:
#def iteration(qc, pos_qubits, I, msg_bits, coin_qubit):#Every red rectangle is iteration
  #U_i
 # num_segments = len(msg_bits) // 2
 # for i in range(num_segments):
 #   m_i = msg_bits[2 * i : 2 * i + 2]  # Two-bit segment
 #   qc.append(U_mi(qc, m_i, I ,pos_qubits, coin_qubit),msg_bits+coin_qubit)
 #   qc.append(inc(coin_qubit),[coin_qubit]+list(range(coin_qubit))) #inc(control)
 #   dec(pos_qubits)

 # return qc

def iteration(qc, pos_qubits, I, msg_qubits, coin_qubit):
    num_segments = len(msg_qubits) // 2
    for i in range(num_segments):
        m_i = ''.join(['1' if qc.qubits[q].index % 2 else '0' for q in range(2*i, 2*i+2)])  # OR just pass classical bits externally

        gate = U_mi(theta_map, m_i, I, pos_qubits, coin_qubit)
        qc.append(gate, [msg_qubits[2*i], msg_qubits[2*i+1], coin_qubit[0]])
        qc.append(inc(len(pos_qubits)), [coin_qubit[0]] + list(pos_qubits))
        qc.append(dec(len(pos_qubits)), [coin_qubit[0]] + list(pos_qubits))

    return qc


In [ ]:
def u_r(n):#Size of position register
  qc = QuantumCircuit(n)
  for i in range(n):
    qc.rx(np.pi / 3, i)

  pass

In [ ]:
def ancilla_system(ancilla, coin, seed=42):
    """
    Builds a U_R block controlled by the coin qubit acting on the ancilla.
    Returns a gate to be appended to (ancilla + coin).
    """
    from qiskit import QuantumCircuit
    from qiskit.quantum_info import random_unitary
    from qiskit.circuit.library import UnitaryGate

    num_anc = ancilla.size
    qc = QuantumCircuit(num_anc + 1)  # ancilla[0...4], coin[5]

    # Apply H to ancilla
    for i in range(num_anc):
        qc.h(i)

    # Apply U_R controlled by coin (index = num_anc), targeting ancilla
    u_r = random_unitary(2 ** num_anc, seed=seed)
    u_r_gate = UnitaryGate(u_r, label="U_R").control(1)
    qc.append(u_r_gate, [num_anc] + list(range(num_anc)))  # [coin] + [ancilla]

    # Final H
    for i in range(num_anc):
        qc.h(i)

    return qc.to_gate(label="UR_Block")

In [ ]:
def build_message_controlled_circuit(message_bits, theta_map):
    """
    Build a quantum circuit that encodes message bits and applies message-controlled unitary on a coin qubit.
    """
    num_stacks = len(message_bits) // 2
    num_message_qubits = 2 * num_stacks  # 2 bits per stack
    msg_register = QuantumRegister(num_message_qubits, name='msg')
    coin_register = QuantumRegister(1, name='coin')
    qc = QuantumCircuit(msg_register, coin_register)

    # # Optional: Initialize message register in a basis state (classical loading)
    # for i, bit in enumerate(message_bits):
    #     if bit == '1':
    #         qc.x(msg_register[i])  # Load classical bit into qubit

    # Apply controlled unitary for each stack
    for i in range(num_stacks):
        msg_indices = [2 * i, 2 * i + 1]
        control_qubits = [msg_register[idx] for idx in msg_indices]
        gate, qubits = U_mi(theta_map, message_bits, i, control_qubits, coin_register[0])
        qc.append(gate, qubits)

    return qc

In [ ]:
def red_box_step(theta_map, msg_bits, step_index, coin_qubit, msg_qubits, pos_qubits):
    """
    One red-box: controlled coin operation followed by coin-controlled position shift.
    """
    from qiskit import QuantumCircuit

    # Extract current 2-bit chunk
    m_i = msg_bits[2 * step_index: 2 * step_index + 2]
    theta = theta_map[m_i]

    # Build controlled coin gate (msg[bit1], msg[bit2]) -> coin
    umi_gate = UnitaryGate(coin_operator(theta), label=f"U_m{step_index}").control(2)
    ctrl_msg = msg_qubits[2 * step_index: 2 * step_index + 2]

    # Create subcircuit with enough qubits (includes msg, coin, pos)
    max_index = max(ctrl_msg + [coin_qubit] + pos_qubits)
    qc = QuantumCircuit(max_index + 1)

    # Apply coin operation
    qc.append(umi_gate, ctrl_msg + [coin_qubit])

    # Apply shift operation
    shift_gate = walk(len(pos_qubits))  # returns gate acting on [coin, pos...]
    qc.append(shift_gate, [coin_qubit] + pos_qubits)

    return qc

In [ ]:
def build_full_hash_circuit(message_bits, theta_map, seed=42):
    if len(message_bits) % 2 != 0:
        message_bits += '0'

    num_segments = len(message_bits) // 2
    num_pos_qubits = int(np.log2(num_segments)) + 1
    num_msg_qubits = len(message_bits)
    num_coin_qubits = 1
    num_ancilla = num_pos_qubits

    total_qubits = num_msg_qubits + num_coin_qubits + num_pos_qubits + num_ancilla
    qc = QuantumCircuit(total_qubits, num_ancilla)

    msg_qubits = list(range(num_msg_qubits))
    coin_qubit = num_msg_qubits
    pos_qubits = list(range(coin_qubit + 1, coin_qubit + 1 + num_pos_qubits))
    ancilla_qubits = list(range(pos_qubits[-1] + 1, total_qubits))

    # Load message bits
    for i, bit in enumerate(message_bits):
        if bit == '1':
            qc.x(msg_qubits[i])

    # Apply one full iteration
    iteration(qc, pos_qubits, 0, message_bits, coin_qubit, theta_map, msg_qubits)

    # Apply ancilla system
    anc_q = QuantumRegister(len(ancilla_qubits), name="anc")
    coin_q = QuantumRegister(1, name="coin")
    pos_q = QuantumRegister(len(pos_qubits), name="pos")
    anc_sys = ancilla_system(anc_q, coin_q, pos_q, seed)

    # Stitching ancilla_system into the main circuit
    qc.compose(anc_sys, qubits=ancilla_qubits + [coin_qubit] + pos_qubits, inplace=True)

    # Measurement
    for i, a in enumerate(ancilla_qubits):
        qc.measure(a, i)

    return qc

In [ ]:
from qiskit import QuantumRegister

def quantum_hash():
  pos_qubits = QuantumRegister(5, "pos")   # Position register (quantum walker)
  coin_qubit = QuantumRegister(1, "coin")
  message_qubits = QuantumRegister(2, "msg")


  qc = QuantumCircuit(pos_qubits, coin_qubit, message_qubits)
  theta_map = { # map for coin operator thetas in U_i
    '00': np.pi / 8,
    '01': np.pi / 6,
    '10': np.pi / 4,
    '11': np.pi / 3
}


  ur_circuit = create_UR_circuit(pos_qubits, coin_qubit, seed=42)
  qc.compose(ur_circuit, inplace=True)

  return qc
    #call iteration after you make it return a circuit


In [ ]:
def quantum_hash():
    from qiskit import QuantumRegister, ClassicalRegister
    pos_qubits = QuantumRegister(5, "pos")
    coin_qubit = QuantumRegister(1, "coin")
    message_qubits = QuantumRegister(2, "msg")
    ancilla = QuantumRegister(5, "anc")
    classical_bits = ClassicalRegister(2, "c")
    qc = QuantumCircuit(pos_qubits, coin_qubit, message_qubits, ancilla, classical_bits)

    qc.h(coin_qubit[0])

    theta_map = {'00': np.pi / 8, '01': np.pi / 6, '10': np.pi / 4, '11': np.pi / 3}

    for i in range(2):
        msg_str = format(i, '02b')
        umi = U_mi(theta_map, msg_str, i, pos_qubits, coin_qubit[0])
        qc.append(umi, [message_qubits[0], message_qubits[1], coin_qubit[0]])

    shift = walk(pos_qubits.size)
    qc.append(shift, [coin_qubit[0]] + list(pos_qubits))

    # Create interference gate with correct number of qubits
    interference_qc = QuantumCircuit(ancilla.size + 1)  # ancilla + coin
    for i in range(ancilla.size):
        interference_qc.h(i)
    u_r = random_unitary(2 ** ancilla.size, seed=42)
    u_r_gate = UnitaryGate(u_r, label="U_R").control(1)
    interference_qc.append(u_r_gate, [ancilla.size] + list(range(ancilla.size)))
    for i in range(ancilla.size):
        interference_qc.h(i)

    # Convert to gate and append
    interference_gate = interference_qc.to_gate(label="UR_Block")
    qc.append(interference_gate, list(ancilla) + [coin_qubit[0]])

    qc.measure(ancilla, classical_bits)

    return qc

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Operator, random_unitary
from qiskit.circuit.library import UnitaryGate
import numpy as np

def inc(n: int):
    qc = QuantumCircuit(n)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i)
    qc.x(n-1)
    return UnitaryGate(Operator(qc), label="Inc").control(1)

def dec(n: int):
    qc = QuantumCircuit(n)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i, ctrl_state='0'*len(controls))
    qc.x(n-1)
    return UnitaryGate(Operator(qc), label="Dec").control(1, ctrl_state='0')

def walk(n: int):
    qc = QuantumCircuit(n + 1)
    qc.append(inc(n), list(range(n+1)))  # First qubit is control
    qc.append(dec(n), list(range(n+1)))  # First qubit is control
    return UnitaryGate(Operator(qc), label="WalkStep")

def coin_operator(theta, label='C'):
    C = np.array([
        [np.cos(theta), np.sin(theta)],
        [np.sin(theta), -np.cos(theta)]
    ])
    return UnitaryGate(C, label=label)

def U_mi(theta_map, m_i, i, coin_qubit):
    theta = theta_map[m_i]
    gate = coin_operator(theta, label=f"U_m{i}")
    return gate.control(2, ctrl_state=m_i)

def ancilla_system(num_ancilla, coin_index, seed=42):
    qc = QuantumCircuit(num_ancilla + 1)
    ancilla_indices = list(range(num_ancilla))
    coin_idx = num_ancilla  # Coin is last qubit

    # Hadamard on ancilla
    for q in ancilla_indices:
        qc.h(q)

    # Controlled unitary
    ur = random_unitary(2**num_ancilla, seed=seed)
    ur_gate = UnitaryGate(ur, label="UR").control(1)
    qc.append(ur_gate, [coin_idx] + ancilla_indices)

    # Hadamard again
    for q in ancilla_indices:
        qc.h(q)

    return qc.to_gate(label="AncillaSystem")

def quantum_hash():
    # Number of qubits
    num_pos = 5
    num_msg = 2
    num_ancilla = 5

    # Create registers
    qr = QuantumRegister(num_pos + 1 + num_msg + num_ancilla, "q")
    cr = ClassicalRegister(num_ancilla, "c")
    qc = QuantumCircuit(qr, cr)

    # Assign indices
    msg_start = 0
    coin_idx = num_msg
    pos_start = coin_idx + 1
    ancilla_start = pos_start + num_pos

    # Initialize coin
    qc.h(coin_idx)

    # Theta mapping
    theta_map = {
        '00': np.pi/8,
        '01': np.pi/6,
        '10': np.pi/4,
        '11': np.pi/3
    }

    # Apply message-controlled operations
    for i in range(2):
        msg_str = format(i, '02b')
        umi_gate = U_mi(theta_map, msg_str, i, coin_idx)
        control1 = msg_start + 0  # First message qubit
        control2 = msg_start + 1  # Second message qubit
        qc.append(umi_gate, [control1, control2, coin_idx])

    # Apply walk step
    walk_gate = walk(num_pos)
    walk_qubits = [coin_idx] + list(range(pos_start, pos_start + num_pos))
    qc.append(walk_gate, walk_qubits)

    # Apply ancilla system
    anc_gate = ancilla_system(num_ancilla, coin_idx)
    anc_qubits = list(range(ancilla_start, ancilla_start + num_ancilla)) + [coin_idx]
    qc.append(anc_gate, anc_qubits)

    # Measure ancilla
    qc.measure(range(ancilla_start, ancilla_start + num_ancilla), range(num_ancilla))

    return qc

# Generate and display the circuit
qc = quantum_hash()
print(qc.draw(fold=-1))  # fold=-1 prevents line wrapping

                                                                         
 q_0: ────────o───────■──────────────────────────────────────────────────
              │       │                                                  
 q_1: ────────o───────o──────────────────────────────────────────────────
      ┌───┐┌──┴───┐┌──┴───┐┌───────────┐┌────────────────┐               
 q_2: ┤ H ├┤ U_m0 ├┤ U_m1 ├┤0          ├┤5               ├───────────────
      └───┘└──────┘└──────┘│           ││                │               
 q_3: ─────────────────────┤1          ├┤                ├───────────────
                           │           ││                │               
 q_4: ─────────────────────┤2          ├┤                ├───────────────
                           │  WalkStep ││                │               
 q_5: ─────────────────────┤3          ├┤                ├───────────────
                           │           ││                │               
 q_6: ─────────────────────┤4         

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Operator, random_unitary
from qiskit.circuit.library import UnitaryGate
import numpy as np

# Define the increment operation
def inc(n: int):
    qc = QuantumCircuit(n)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i)
    qc.x(n-1)
    return UnitaryGate(Operator(qc), label="Inc").control(1)

# Define the decrement operation
def dec(n: int):
    qc = QuantumCircuit(n)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i, ctrl_state='0'*len(controls))
    qc.x(n-1)
    return UnitaryGate(Operator(qc), label="Dec").control(1, ctrl_state='0')

# Define the walk step (shift operation)
def walk(n: int):
    qc = QuantumCircuit(n + 1)
    qc.append(inc(n), list(range(n+1)))  # First qubit is control (coin)
    qc.append(dec(n), list(range(n+1)))  # First qubit is control (coin)
    return UnitaryGate(Operator(qc), label="Shift")

# Define the coin operator
def create_coin_operator(theta, label='C'):
    C = np.array([
        [np.cos(theta), np.sin(theta)],
        [np.sin(theta), -np.cos(theta)]
    ])
    return UnitaryGate(C, label=label)

# Main quantum hash circuit function
def quantum_hash_circuit(message_bits, n=1, q=1, max_pos_qubits=8, max_anc_qubits=8):
    # Validate message length (fixed at 2 bits)
    msg_length = 2
    if len(message_bits) != msg_length:
        raise ValueError(f"Message must be exactly 2 bits, got {len(message_bits)} bits")

    # Validate position and ancilla qubit counts
    if n > max_pos_qubits:
        raise ValueError(f"Number of position qubits ({n}) exceeds maximum allowed ({max_pos_qubits})")
    if q > max_anc_qubits:
        raise ValueError(f"Number of ancilla qubits ({q}) exceeds maximum allowed ({max_anc_qubits})")

    # Create registers
    pos_reg = QuantumRegister(n, 'pos')  # n position qubits (max 8)
    coin_reg = QuantumRegister(1, 'coin')  # 1 coin qubit
    msg_reg = QuantumRegister(msg_length, 'msg')  # 2 message qubits
    anc_reg = QuantumRegister(q, 'anc')  # q ancilla qubits (max 8)
    cr = ClassicalRegister(q, 'c')  # Classical register for measurement

    qc = QuantumCircuit(pos_reg, coin_reg, msg_reg, anc_reg, cr)

    # Initialize message bits
    for i, bit in enumerate(message_bits):
        if bit == '1':
            qc.x(msg_reg[i])

    # Theta mapping for 2-bit message
    theta_map = {
        '00': np.pi/8,
        '01': np.pi/6,
        '10': np.pi/4,
        '11': np.pi/3
    }

    # Get theta for the current message
    theta = theta_map[message_bits]

    # First block: U_m0 and Shift
    # U_m0 on coin, controlled by the 2 message qubits
    U_m0 = create_coin_operator(theta, label='U_m0').control(msg_length)
    qc.append(U_m0, list(msg_reg) + [coin_reg[0]])

    # Shift on position qubits, controlled by coin
    shift_gate = walk(n)
    qc.append(shift_gate, [coin_reg[0]] + list(pos_reg))

    # Second block: U_m1 and Shift
    # U_m1 on coin, controlled by the 2 message qubits
    U_m1 = create_coin_operator(theta, label='U_m1').control(msg_length)
    qc.append(U_m1, list(msg_reg) + [coin_reg[0]])

    # Shift on position qubits, controlled by coin
    qc.append(shift_gate, [coin_reg[0]] + list(pos_reg))

    # Ancilla system: H -> U_R -> H -> Measure
    # Hadamard on ancilla
    qc.h(anc_reg)

    # U_R on ancilla, controlled by coin
    ur = random_unitary(2**q, seed=42)
    ur_gate = UnitaryGate(ur, label='U_R').control(1)
    qc.append(ur_gate, [coin_reg[0]] + list(anc_reg))

    # Hadamard on ancilla
    qc.h(anc_reg)

    # Measurement of ancilla
    qc.measure(anc_reg, cr)

    return qc

# Example usage with different position and ancilla qubit counts
# Test with n=3 position qubits, q=2 ancilla qubits
message = '01'
qc = quantum_hash_circuit(message_bits=message, n=3, q=2)
print("Circuit with 3 position qubits and 2 ancilla qubits:")
print(qc.draw(fold=-1))

# Test with maximum position and ancilla qubits (n=8, q=8)
qc_max = quantum_hash_circuit(message_bits=message, n=8, q=8)
print("\nCircuit with 8 position qubits and 8 ancilla qubits:")
print(qc_max.draw(fold=-1))

Circuit with 3 position qubits and 2 ancilla qubits:
                    ┌────────┐        ┌────────┐                   
pos_0: ─────────────┤1       ├────────┤1       ├───────────────────
                    │        │        │        │                   
pos_1: ─────────────┤2       ├────────┤2       ├───────────────────
                    │  Shift │        │  Shift │                   
pos_2: ─────────────┤3       ├────────┤3       ├───────────────────
            ┌──────┐│        │┌──────┐│        │                   
 coin: ─────┤ U_m0 ├┤0       ├┤ U_m1 ├┤0       ├───■───────────────
            └──┬───┘└────────┘└──┬───┘└────────┘   │               
msg_0: ────────■─────────────────■─────────────────┼───────────────
       ┌───┐   │                 │                 │               
msg_1: ┤ X ├───■─────────────────■─────────────────┼───────────────
       ├───┤                                    ┌──┴───┐┌───┐┌─┐   
anc_0: ┤ H ├────────────────────────────────────┤0     ├┤ H ├┤M

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Operator, random_unitary
from qiskit.circuit.library import UnitaryGate
import numpy as np
import math

# Define the increment operation
def inc(n: int):
    qc = QuantumCircuit(n)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i)
    qc.x(n-1)
    return UnitaryGate(Operator(qc), label="Inc").control(1)

# Define the decrement operation
def dec(n: int):
    qc = QuantumCircuit(n)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i, ctrl_state='0'*len(controls))
    qc.x(n-1)
    return UnitaryGate(Operator(qc), label="Dec").control(1, ctrl_state='0')

# Define the walk step (shift operation)
def walk(n: int):
    qc = QuantumCircuit(n + 1)
    qc.append(inc(n), list(range(n+1)))  # First qubit is control (coin)
    qc.append(dec(n), list(range(n+1)))  # First qubit is control (coin)
    return UnitaryGate(Operator(qc), label="Shift")

# Define the coin operator
def create_coin_operator(theta, label='U_m'):
    C = np.array([
        [np.cos(theta), np.sin(theta)],
        [np.sin(theta), -np.cos(theta)]
    ])
    return UnitaryGate(C, label=label)

# Main quantum hash circuit function
def quantum_hash_circuit(message_bits, max_pos_qubits=8, max_anc_qubits=8, max_msg_length=256):
    # Validate message length
    msg_length = len(message_bits)
    if msg_length > max_msg_length:
        raise ValueError(f"Message length ({msg_length}) exceeds maximum allowed ({max_msg_length})")

    # Calculate number of position qubits: log_2(L), where L is the message length
    n_pos = math.ceil(math.log2(msg_length)) if msg_length > 0 else 1  # Ensure at least 1 qubit
    n_pos = min(n_pos, max_pos_qubits)  # Cap at maximum allowed

    # Number of ancilla qubits must equal number of position qubits
    n_anc = n_pos
    if n_anc > max_anc_qubits:
        raise ValueError(f"Number of ancilla qubits ({n_anc}) exceeds maximum allowed ({max_anc_qubits})")

    # Divide message into packets of 2 bits
    packet_size = 2
    packets = []
    for i in range(0, msg_length, packet_size):
        packet = message_bits[i:i + packet_size]
        # Pad the last packet with a '0' if necessary
        if len(packet) < packet_size:
            packet = packet + '0' * (packet_size - len(packet))
        packets.append(packet)

    num_packets = len(packets)  # Number of red box iterations

    # Create registers
    pos_reg = QuantumRegister(n_pos, 'pos')  # Position qubits (log_2(L), max 8)
    coin_reg = QuantumRegister(1, 'coin')  # 1 coin qubit
    msg_reg = QuantumRegister(2, 'msg')  # Exactly 2 message qubits
    anc_reg = QuantumRegister(n_anc, 'anc')  # Ancilla qubits (equal to position qubits, max 8)
    cr = ClassicalRegister(n_anc, 'c')  # Classical register for measurement

    qc = QuantumCircuit(pos_reg, coin_reg, msg_reg, anc_reg, cr)

    # Theta mapping for 2-bit packets
    theta_map = {
        '00': np.pi/8,
        '01': np.pi/6,
        '10': np.pi/4,
        '11': np.pi/3
    }

    # Process each packet using the 2 message qubits
    for i, packet in enumerate(packets):
        # Initialize the 2 message qubits with the current packet's bits
        for j, bit in enumerate(packet):
            if bit == '1':
                qc.x(msg_reg[j])

        # Get the theta value for this packet
        theta = theta_map[packet]

        # Apply U_m on coin, controlled by the 2 message qubits
        U_m = create_coin_operator(theta, label=f'U_m{i}')
        U_m_controlled = U_m.control(2)  # Controlled by 2 qubits
        qc.append(U_m_controlled, list(msg_reg) + [coin_reg[0]])

        # Shift on position qubits, controlled by coin
        shift_gate = walk(n_pos)
        qc.append(shift_gate, [coin_reg[0]] + list(pos_reg))

        # Reset the message qubits for the next packet
        for j, bit in enumerate(packet):
            if bit == '1':
                qc.x(msg_reg[j])

    # Ancilla system: H -> U_R -> H -> Measure
    # Hadamard on ancilla
    qc.h(anc_reg)

    # U_R on ancilla, controlled by coin
    ur = random_unitary(2**n_anc, seed=42)
    ur_gate = UnitaryGate(ur, label='U_R').control(1)
    qc.append(ur_gate, [coin_reg[0]] + list(anc_reg))

    # Hadamard on ancilla
    qc.h(anc_reg)

    # Measurement of ancilla
    qc.measure(anc_reg, cr)

    return qc

# Test with a 4-bit message (L = 4, log_2(4) = 2 position qubits, 2 packets: '10', '11')
message_4bit = '1011'
qc_4bit = quantum_hash_circuit(message_bits=message_4bit)
print("Circuit with 4-bit message (log_2(4) = 2 position qubits, 2 ancilla qubits, 2 packets, 2 red box iterations):")
print(qc_4bit.draw(fold=-1))

# Test with a 5-bit message (L = 5, log_2(5) ≈ 2.32 → 3 position qubits, 3 packets: '10', '11', '00' (padded))
message_5bit = '10110'
qc_5bit = quantum_hash_circuit(message_bits=message_5bit)
print("\nCircuit with 5-bit message (log_2(5) = 3 position qubits, 3 ancilla qubits, 3 packets, 3 red box iterations):")
print(qc_5bit.draw(fold=-1))

# Test with a 256-bit message (L = 256, log_2(256) = 8 position qubits, 128 packets)
message_256bit = '1' * 156
qc_256bit = quantum_hash_circuit(message_bits=message_256bit)
print("\nCircuit with 256-bit message (log_2(256) = 8 position qubits, 8 ancilla qubits, 128 packets, 128 red box iterations):")
print(qc_256bit.draw(fold=-1))

Circuit with 4-bit message (log_2(4) = 2 position qubits, 2 ancilla qubits, 2 packets, 2 red box iterations):
                    ┌────────┐             ┌────────┐                   
pos_0: ─────────────┤1       ├─────────────┤1       ├───────────────────
                    │        │             │        │                   
pos_1: ─────────────┤2 Shift ├─────────────┤2 Shift ├───────────────────
            ┌──────┐│        │     ┌──────┐│        │                   
 coin: ─────┤ U_m0 ├┤0       ├─────┤ U_m1 ├┤0       ├───■───────────────
       ┌───┐└──┬───┘└─┬───┬──┘┌───┐└──┬───┘└─┬───┬──┘   │               
msg_0: ┤ X ├───■──────┤ X ├───┤ X ├───■──────┤ X ├──────┼───────────────
       └───┘   │      ├───┤   └───┘   │      ├───┤      │               
msg_1: ────────■──────┤ X ├───────────■──────┤ X ├──────┼───────────────
       ┌───┐          └───┘                  └───┘   ┌──┴───┐┌───┐┌─┐   
anc_0: ┤ H ├─────────────────────────────────────────┤0     ├┤ H ├┤M├───
       ├───┤  

KeyboardInterrupt: 

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Operator, random_unitary
from qiskit.circuit.library import UnitaryGate
import numpy as np
import math

# Define the increment operation
def inc(n: int):
    qc = QuantumCircuit(n)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i)
    qc.x(n-1)
    return UnitaryGate(Operator(qc), label="Inc").control(1)

# Define the decrement operation
def dec(n: int):
    qc = QuantumCircuit(n)
    for i in range(n-1):
        controls = list(range(i+1, n))
        qc.mcx(controls, i, ctrl_state='0'*len(controls))
    qc.x(n-1)
    return UnitaryGate(Operator(qc), label="Dec").control(1, ctrl_state='0')

# Define the walk step (shift operation)
def walk(n: int):
    qc = QuantumCircuit(n + 1)
    qc.append(inc(n), list(range(n+1)))  # First qubit is control (coin)
    qc.append(dec(n), list(range(n+1)))  # First qubit is control (coin)
    return UnitaryGate(Operator(qc), label="Shift")

# Define the coin operator
def create_coin_operator(theta, label='U_m'):
    C = np.array([
        [np.cos(theta), np.sin(theta)],
        [np.sin(theta), -np.cos(theta)]
    ])
    return UnitaryGate(C, label=label)

# Main quantum hash circuit function
def quantum_hash_circuit(message_bits, max_pos_qubits=8, max_anc_qubits=8, max_msg_length=256):
    # Validate message length
    msg_length = len(message_bits)
    if msg_length > max_msg_length:
        raise ValueError(f"Message length ({msg_length}) exceeds maximum allowed ({max_msg_length})")

    # Calculate number of position qubits: log_2(L), where L is the message length
    n_pos = math.ceil(math.log2(msg_length)) if msg_length > 0 else 1  # Ensure at least 1 qubit
    n_pos = min(n_pos, max_pos_qubits)  # Cap at maximum allowed

    # Number of ancilla qubits must equal number of position qubits
    n_anc = n_pos
    if n_anc > max_anc_qubits:
        raise ValueError(f"Number of ancilla qubits ({n_anc}) exceeds maximum allowed ({max_anc_qubits})")

    # Divide message into packets of 2 bits
    packet_size = 2
    packets = []
    for i in range(0, msg_length, packet_size):
        packet = message_bits[i:i + packet_size]
        # Pad the last packet with a '0' if necessary
        if len(packet) < packet_size:
            packet = packet + '0' * (packet_size - len(packet))
        packets.append(packet)

    num_packets = len(packets)  # Number of red box iterations

    # Create registers
    pos_reg = QuantumRegister(n_pos, 'pos')  # Position qubits (log_2(L), max 8)
    coin_reg = QuantumRegister(1, 'coin')  # 1 coin qubit
    msg_reg = QuantumRegister(2, 'msg')  # Exactly 2 message qubits
    anc_reg = QuantumRegister(n_anc, 'anc')  # Ancilla qubits (equal to position qubits, max 8)
    cr = ClassicalRegister(n_anc, 'c')  # Classical register for measurement

    qc = QuantumCircuit(pos_reg, coin_reg, msg_reg, anc_reg, cr)

    # Theta mapping for 2-bit packets
    theta_map = {
        '00': np.pi/8,
        '01': np.pi/6,
        '10': np.pi/4,
        '11': np.pi/3
    }

    # Process each packet using the 2 message qubits
    for i, packet in enumerate(packets):
        # Initialize the 2 message qubits with the current packet's bits
        for j, bit in enumerate(packet):
            if bit == '1':
                qc.x(msg_reg[j])

        # Get the theta value for this packet
        theta = theta_map[packet]

        # Apply U_m on coin, controlled by the 2 message qubits
        U_m = create_coin_operator(theta, label=f'U_m{i}')
        U_m_controlled = U_m.control(2)  # Controlled by 2 qubits
        qc.append(U_m_controlled, list(msg_reg) + [coin_reg[0]])

        # Shift on position qubits, controlled by coin
        shift_gate = walk(n_pos)
        qc.append(shift_gate, [coin_reg[0]] + list(pos_reg))

        # Reset the message qubits for the next packet
        for j, bit in enumerate(packet):
            if bit == '1':
                qc.x(msg_reg[j])

    # Ancilla system: H -> U_R -> H -> Measure
    # Hadamard on ancilla
    qc.h(anc_reg)

    # U_R on position qubits, controlled by ancilla qubits
    ur = random_unitary(2**n_pos, seed=42)
    ur_gate = UnitaryGate(ur, label='U_R').control(n_anc)  # Controlled by n_anc ancilla qubits
    qc.append(ur_gate, list(anc_reg) + list(pos_reg))

    # Hadamard on ancilla
    qc.h(anc_reg)

    # Measurement of ancilla
    qc.measure(anc_reg, cr)

    return qc

# Test with a 4-bit message (L = 4, log_2(4) = 2 position qubits, 2 ancilla qubits, 2 packets: '10', '11')
message_4bit = '1011'
qc_4bit = quantum_hash_circuit(message_bits=message_4bit)
print("Circuit with 4-bit message (log_2(4) = 2 position qubits, 2 ancilla qubits, 2 packets, 2 red box iterations):")
print(qc_4bit.draw(fold=-1))

# Test with a 5-bit message (L = 5, log_2(5) ≈ 2.32 → 3 position qubits, 3 ancilla qubits, 3 packets: '10', '11', '00' (padded))
message_5bit = '10110'
qc_5bit = quantum_hash_circuit(message_bits=message_5bit)
print("\nCircuit with 5-bit message (log_2(5) = 3 position qubits, 3 ancilla qubits, 3 packets, 3 red box iterations):")
print(qc_5bit.draw(fold=-1))

Circuit with 4-bit message (log_2(4) = 2 position qubits, 2 ancilla qubits, 2 packets, 2 red box iterations):
                    ┌────────┐             ┌────────┐┌──────┐           
pos_0: ─────────────┤1       ├─────────────┤1       ├┤0     ├───────────
                    │        │             │        ││  U_R │           
pos_1: ─────────────┤2 Shift ├─────────────┤2 Shift ├┤1     ├───────────
            ┌──────┐│        │     ┌──────┐│        │└──┬───┘           
 coin: ─────┤ U_m0 ├┤0       ├─────┤ U_m1 ├┤0       ├───┼───────────────
       ┌───┐└──┬───┘└─┬───┬──┘┌───┐└──┬───┘└─┬───┬──┘   │               
msg_0: ┤ X ├───■──────┤ X ├───┤ X ├───■──────┤ X ├──────┼───────────────
       └───┘   │      ├───┤   └───┘   │      ├───┤      │               
msg_1: ────────■──────┤ X ├───────────■──────┤ X ├──────┼───────────────
       ┌───┐          └───┘                  └───┘      │    ┌───┐┌─┐   
anc_0: ┤ H ├────────────────────────────────────────────■────┤ H ├┤M├───
       ├───┤  

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import random_unitary
from qiskit.circuit.library import UnitaryGate
import numpy as np

def ancilla_UR_gate(num_ancilla, num_pos, seed=42):
    total_qubits = num_ancilla + num_pos
    qc = QuantumCircuit(total_qubits)

    # Apply H to ancilla
    for i in range(num_ancilla):
        qc.h(i)

    # Create a random unitary acting on the position register
    ur = random_unitary(2 ** num_pos, seed=seed)
    ur_gate = UnitaryGate(ur, label="U_R").control(num_ancilla)

    # Controls = ancilla (first part), Targets = pos (second part)
    controls = list(range(num_ancilla))
    targets = list(range(num_ancilla, total_qubits))
    qc.append(ur_gate, controls + targets)

    # Final H on ancilla
    for i in range(num_ancilla):
        qc.h(i)

    return qc.to_gate(label="UR_Block")

# === Build the test circuit ===
def build_final_test_circuit():
    anc = QuantumRegister(5, 'anc')
    pos = QuantumRegister(5, 'pos')
    classical = ClassicalRegister(5, 'c')
    qc = QuantumCircuit(anc, pos, classical)

    ur_gate = ancilla_UR_gate(5, 5)
    qc.append(ur_gate, list(anc) + list(pos))

    qc.measure(anc, classical)
    return qc

qc = build_final_test_circuit()
print(qc.draw('text'))

# # === Optional: Simulate it ===
# backend = Aer.get_backend("qasm_simulator")
# result = execute(qc, backend, shots=512).result()
# counts = result.get_counts()
# print("\nHash output (ancilla measurement):")
# print(counts)

       ┌───────────┐┌─┐            
anc_0: ┤0          ├┤M├────────────
       │           │└╥┘┌─┐         
anc_1: ┤1          ├─╫─┤M├─────────
       │           │ ║ └╥┘┌─┐      
anc_2: ┤2          ├─╫──╫─┤M├──────
       │           │ ║  ║ └╥┘┌─┐   
anc_3: ┤3          ├─╫──╫──╫─┤M├───
       │           │ ║  ║  ║ └╥┘┌─┐
anc_4: ┤4          ├─╫──╫──╫──╫─┤M├
       │  UR_Block │ ║  ║  ║  ║ └╥┘
pos_0: ┤5          ├─╫──╫──╫──╫──╫─
       │           │ ║  ║  ║  ║  ║ 
pos_1: ┤6          ├─╫──╫──╫──╫──╫─
       │           │ ║  ║  ║  ║  ║ 
pos_2: ┤7          ├─╫──╫──╫──╫──╫─
       │           │ ║  ║  ║  ║  ║ 
pos_3: ┤8          ├─╫──╫──╫──╫──╫─
       │           │ ║  ║  ║  ║  ║ 
pos_4: ┤9          ├─╫──╫──╫──╫──╫─
       └───────────┘ ║  ║  ║  ║  ║ 
  c: 5/══════════════╩══╩══╩══╩══╩═
                     0  1  2  3  4 


In [ ]:
qc = quantum_hash()
print(qc.draw('text'))